State Dataset
-  2018 CO2 emissions per capita 
-  2015 governor political party 
-  Membership in the United States Climate alliance in it’s inauguration 
-  2018 State Partisan Index



https://www.eia.gov/environment/emissions/state/ - state ghg emissions

https://github.com/fivethirtyeight/data/blob/master/partisan-lean/fivethirtyeight_partisan_lean_STATES.csv - state partisan index

http://www.usclimatealliance.org/governors-1 - membership in US climate alliance

https://ballotpedia.org/Partisan_composition_of_governors - party of governors



In [1]:
from os import path
import requests
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as Soup
import pandas as pd
from pandas import DataFrame
import numpy as np
DATA_DIR = '/Users/joeskvarna/Desktop/Thesis/Data'

# 1. State CO2 per capita

Import file and take a look

In [2]:
state_ghg = pd.read_csv(path.join(DATA_DIR, 'state_ghg.csv'), header = None)
state_ghg.head(6)

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,Table 5. Per capita energy-related carbon diox...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,metric tons of energy-related carbon dioxide p...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Change,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(2000–2018),NaN
5,State,2000.0,2001.0,2002.0,2003.0,2004.0,2005.0,2006.0,2007.0,2008.0,...,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,Percent,Absolute


I want to remove the first 5 rows. But I want to be able to run this whole jupyter notebook over and over and get the same output. So I will remove the first 5 rows inside of an if statement that only runs if there are null values in the first row.

We can see that the rows we want don't have null values

In [3]:
state_ghg.isnull().sum(axis = 1)[:10]

0    21
1    21
2    22
3    21
4    21
5     0
6     0
7     0
8     0
9     0
dtype: int64

remove rows, set header, and reset index index

In [4]:
if state_ghg.isnull().any(axis = 1)[0]:
    state_ghg.drop(labels = range(5), inplace = True)
    state_ghg.columns = state_ghg.iloc[0]
    state_ghg = state_ghg[1:]
    state_ghg.reset_index(drop = True, inplace = True)
state_ghg.head()

5,State,2000.0,2001.0,2002.0,2003.0,2004.0,2005.0,2006.0,2007.0,2008.0,...,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,Percent,Absolute
0,Alabama,32.1,30.0,31.1,31.1,31.5,31.6,31.7,31.7,29.8,...,27.2,25.6,25.1,25.5,24.7,23.6,22.4,23.2,-27.9%,-8.9
1,Alaska,70.7,68.6,68.0,67.5,71.1,72.3,68.1,65.0,57.7,...,52.3,50.3,47.1,46.8,48.1,45.7,46.4,47.6,-32.7%,-23.1
2,Arizona,16.7,16.8,16.3,16.3,17.2,16.6,16.6,16.6,16.3,...,15.0,14.5,14.9,14.4,13.9,13.1,12.8,13.1,-21.7%,-3.6
3,Arkansas,23.8,23.4,22.7,22.8,22.8,21.7,22.0,22.3,22.4,...,23.0,22.5,23.2,23.3,19.9,20.9,21.5,23.6,-0.8%,-0.2
4,California,11.1,11.1,10.9,10.5,10.9,10.7,10.9,11.0,10.4,...,9.1,9.1,9.1,8.9,9.0,9.0,9.0,9.0,-18.4%,-2.0


Change all columns headers to data type string

In [5]:
state_ghg.columns = [str(i) for i in state_ghg.columns]
state_ghg.columns

Index(['State', '2000.0', '2001.0', '2002.0', '2003.0', '2004.0', '2005.0',
       '2006.0', '2007.0', '2008.0', '2009.0', '2010.0', '2011.0', '2012.0',
       '2013.0', '2014.0', '2015.0', '2016.0', '2017.0', '2018.0', 'Percent',
       'Absolute'],
      dtype='object')

Remove '.0' from year columns and rename last two columns. Also will remove the header on the index column

In [6]:
if state_ghg.columns[1] != '2000':     
    state_list = ['state']
    for i in state_ghg.columns[1:-2]:
        i = i[:-2]
        state_list.append(i)
    state_list.append('percent change')
    state_list.append('absolute change')
    state_ghg.columns = state_list
state_ghg.columns

Index(['state', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007',
       '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016',
       '2017', '2018', 'percent change', 'absolute change'],
      dtype='object')

In [7]:
state_ghg.head()

,state,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2011,2012,2013,2014,2015,2016,2017,2018,percent change,absolute change
0,Alabama,32.1,30.0,31.1,31.1,31.5,31.6,31.7,31.7,29.8,...,27.2,25.6,25.1,25.5,24.7,23.6,22.4,23.2,-27.9%,-8.9
1,Alaska,70.7,68.6,68.0,67.5,71.1,72.3,68.1,65.0,57.7,...,52.3,50.3,47.1,46.8,48.1,45.7,46.4,47.6,-32.7%,-23.1
2,Arizona,16.7,16.8,16.3,16.3,17.2,16.6,16.6,16.6,16.3,...,15.0,14.5,14.9,14.4,13.9,13.1,12.8,13.1,-21.7%,-3.6
3,Arkansas,23.8,23.4,22.7,22.8,22.8,21.7,22.0,22.3,22.4,...,23.0,22.5,23.2,23.3,19.9,20.9,21.5,23.6,-0.8%,-0.2
4,California,11.1,11.1,10.9,10.5,10.9,10.7,10.9,11.0,10.4,...,9.1,9.1,9.1,8.9,9.0,9.0,9.0,9.0,-18.4%,-2.0


Want to eliminate some of the last rows. See below.

In [8]:
state_ghg.tail(8)

,state,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2011,2012,2013,2014,2015,2016,2017,2018,percent change,absolute change
50,Wyoming,128.1,128.4,124.7,127.9,126.2,123.6,123.0,124.7,123.0,...,113.4,115.4,117.9,112.7,108.9,104.4,107.8,109.9,-14.2%,-18.2
51,Average all states,20.7,20.1,20.1,20.1,20.3,20.2,19.7,19.8,19.0,...,17.5,16.6,16.9,17.0,16.4,16.0,15.8,16.2,-21.7%,-4.5
52,Source: U.S. Energy Information Administratio...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Remove everything below index 51. I can do this simply by removing any rows that have null values

In [9]:
state_ghg.dropna(axis = 0, how = 'any', inplace = True)
state_ghg.tail()

,state,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2011,2012,2013,2014,2015,2016,2017,2018,percent change,absolute change
47,Washington,13.2,13.0,11.8,12.1,12.1,12.1,11.5,12.2,11.5,...,9.8,9.5,10.4,10.0,10.1,10.6,10.4,10.2,-22.6%,-3.0
48,West Virginia,63.6,57.9,65.1,63.0,61.0,62.1,61.9,62.9,60.4,...,53.1,50.0,51.3,54.5,50.4,52.1,50.4,49.8,-21.7%,-13.8
49,Wisconsin,20.1,19.6,19.7,19.2,19.5,20.0,18.5,18.7,18.6,...,17.2,15.9,17.7,17.7,17.4,16.7,17.2,17.6,-12.7%,-2.6
50,Wyoming,128.1,128.4,124.7,127.9,126.2,123.6,123.0,124.7,123.0,...,113.4,115.4,117.9,112.7,108.9,104.4,107.8,109.9,-14.2%,-18.2
51,Average all states,20.7,20.1,20.1,20.1,20.3,20.2,19.7,19.8,19.0,...,17.5,16.6,16.9,17.0,16.4,16.0,15.8,16.2,-21.7%,-4.5


Just keep the state and year 2018 data

In [10]:
state_ghg = state_ghg[['state', '2018']]
state_ghg.rename(columns = {'2018': 'CO2_per_capita'}, inplace = True)
state_ghg.head(10)

,state,CO2_per_capita
0,Alabama,23.2
1,Alaska,47.6
2,Arizona,13.1
3,Arkansas,23.6
4,California,9.0
5,Colorado,15.9
6,Connecticut,10.5
7,Delaware,13.8
8,District of Columbia,4.1
9,Florida,11.4


# 2. Partisan Lean of States

open and checkout file

In [11]:
partisan_lean = pd.read_csv(path.join(DATA_DIR, 'fivethirtyeight_partisan_lean_STATES.csv'))
partisan_lean.head()

,state,2018
0,Alabama,R+27
1,Alaska,R+15
2,Arizona,R+9
3,Arkansas,R+24
4,California,D+24


split the partisan number and party up

In [12]:
if len(partisan_lean.columns) == 2:
    pd.options.mode.chained_assignment = None
    partisan_lean['party'] = partisan_lean['2018'].str.split('+').str[0]
    partisan_lean['2018'] = partisan_lean['2018'].str.split('+').str[1]
partisan_lean.head()

,state,2018,party
0,Alabama,27,R
1,Alaska,15,R
2,Arizona,9,R
3,Arkansas,24,R
4,California,24,D


create a column called sign. We will make Republicans positive and Democrats negative.

In [13]:
partisan_lean['sign'] = np.nan
partisan_lean.loc[partisan_lean['party'] == 'R', 'sign'] = '+'
partisan_lean.loc[partisan_lean['party'] == 'D', 'sign'] = '-'
partisan_lean.head()

,state,2018,party,sign
0,Alabama,27,R,+
1,Alaska,15,R,+
2,Arizona,9,R,+
3,Arkansas,24,R,+
4,California,24,D,-


combine the sign column and 2018 column to make a numeric partisanship column. Again negative indicates Democratic leaning while positive indicates Republican.

In [14]:
if len(partisan_lean.columns) >=3:
    partisan_lean['partisanship'] = partisan_lean['sign'] + partisan_lean['2018']
    partisan_lean['partisanship'] = partisan_lean['partisanship'].astype(int)
    partisan_lean = partisan_lean[['state', 'partisanship']]
partisan_lean.head(10)

,state,partisanship
0,Alabama,27
1,Alaska,15
2,Arizona,9
3,Arkansas,24
4,California,-24
5,Colorado,-1
6,Connecticut,-11
7,Delaware,-14
8,Florida,5
9,Georgia,12


# 3. Combine 1 and 2

In [15]:
state_ghg.head()

,state,CO2_per_capita
0,Alabama,23.2
1,Alaska,47.6
2,Arizona,13.1
3,Arkansas,23.6
4,California,9.0


In [16]:
partisan_lean.head()

,state,partisanship
0,Alabama,27
1,Alaska,15
2,Arizona,9
3,Arkansas,24
4,California,-24


In [17]:
state_partisan = pd.merge(state_ghg, partisan_lean, how='left')
state_partisan.head()

,state,CO2_per_capita,partisanship
0,Alabama,23.2,27.0
1,Alaska,47.6,15.0
2,Arizona,13.1,9.0
3,Arkansas,23.6,24.0
4,California,9.0,-24.0


Remove district of Columbia and Average of all states. DC is included in our dataset and average will not be used for analysis. They both have null values in the partisanship column so we can remove these rows based on having null values

In [18]:
state_partisan.dropna(axis = 0, how = 'any', inplace = True)
state_partisan.head()

,state,CO2_per_capita,partisanship
0,Alabama,23.2,27.0
1,Alaska,47.6,15.0
2,Arizona,13.1,9.0
3,Arkansas,23.6,24.0
4,California,9.0,-24.0


# 4. Add Governor Name and Party

Scrape data from website

In [19]:
url = 'https://ballotpedia.org/Partisan_composition_of_governors'
def scraper(url):
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()

    page_soup = Soup(webpage, 'html.parser')
    return page_soup
page_soup = scraper(url)

Set up our row parser after investigating the html elements

In [99]:
tables = page_soup.find_all('table')
table = tables[1]
rows = table.find_all('tr')
def parse_row(row):
    """
    Take in a td tag and get the data out of it in the form of a list of
    strings
    """
    #return the state
    data_row_state = row.find_all('td')[0]
    a_state = data_row_state.find_all('a')
    state = (str(a_state).split('>')[1])[12:-3]
    
    #return the governor name
    data_row_name = row.find_all('td')[1]
    a_name = data_row_name.find_all('a')
    name = (str(a_name).split('>')[1])[:-3]
    
    #return the party
    data_row_party = row.find_all('td')[2]
    party = (str(data_row_party).split('>')[1])[:-4]
    
    return [state, name, party]


check row parser on a single row

In [100]:
parse_row(rows[1])

['Alabama', 'Kay Ivey', 'Republican ']

Create dataframe from rows using the row parser. Note that the first row is the title and we don't want it

In [104]:
list_of_parsed_rows = [parse_row(row) for row in rows[1:]]
governor_party = DataFrame(list_of_parsed_rows)
governor_party.head()

,0,1,2
0,Alabama,Kay Ivey,Republican
1,Alaska,Mike Dunleavy,Republican
2,American Samoa,Lemanu Palepoi Mauga,Democratic
3,Arizona,Doug Ducey,Republican
4,Arkansas,Asa Hutchinson,Republican


rename columns in df

In [106]:
governor_party.rename(columns = {0:'state', 1:'governor', 2:'governor_party'}, inplace = True)
governor_party.head()

,state,governor,governor_party
0,Alabama,Kay Ivey,Republican
1,Alaska,Mike Dunleavy,Republican
2,American Samoa,Lemanu Palepoi Mauga,Democratic
3,Arizona,Doug Ducey,Republican
4,Arkansas,Asa Hutchinson,Republican


Merge above (governor_party) and our merged datatset (state_partisan)

In [107]:
state_partisan.head()

,state,CO2_per_capita,partisanship
0,Alabama,23.2,27.0
1,Alaska,47.6,15.0
2,Arizona,13.1,9.0
3,Arkansas,23.6,24.0
4,California,9.0,-24.0


In [113]:
state_dataset = pd.merge(state_partisan, governor_party, how='left')
state_dataset.head(10)

,state,CO2_per_capita,partisanship,governor,governor_party
0,Alabama,23.2,27.0,Kay Ivey,Republican
1,Alaska,47.6,15.0,Mike Dunleavy,Republican
2,Arizona,13.1,9.0,Doug Ducey,Republican
3,Arkansas,23.6,24.0,Asa Hutchinson,Republican
4,California,9.0,-24.0,Gavin Newsom,Democratic
5,Colorado,15.9,-1.0,Jared Polis,Democratic
6,Connecticut,10.5,-11.0,Ned Lamont,Democratic
7,Delaware,13.8,-14.0,John C. Carney Jr.,Democratic
8,Florida,11.4,5.0,Ron DeSantis,Republican
9,Georgia,13.6,12.0,Brian Kemp,Republican


Check if we lost any states in merge

In [112]:
len(state_partisan['state']) == len(state_dataset['state'])

True

All good

# 5. Membership in Climate Alliance

create nan column

In [115]:
state_dataset['climate_alliance'] = np.nan
state_dataset.head()

,state,CO2_per_capita,partisanship,governor,governor_party,climate_alliance
0,Alabama,23.2,27.0,Kay Ivey,Republican,NaN
1,Alaska,47.6,15.0,Mike Dunleavy,Republican,NaN
2,Arizona,13.1,9.0,Doug Ducey,Republican,NaN
3,Arkansas,23.6,24.0,Asa Hutchinson,Republican,NaN
4,California,9.0,-24.0,Gavin Newsom,Democratic,NaN


This last row will be very easy to fill in manually on a csv file, it will probably take 5 minutes. I will save this dataset, fill in the last column mnually, then reimport it

In [116]:
state_dataset.to_csv('/Users/joeskvarna/Desktop/Thesis/Data/state_dataset_fillin.csv', index = False)

In [124]:
state_dataset = pd.read_csv(path.join(DATA_DIR, 'state_dataset_reimport.csv'))
state_dataset.head()

,state,CO2_per_capita,partisanship,governor,governor_party,climate_alliance
0,Alabama,23.2,27.0,Kay Ivey,Republican,NaN
1,Alaska,47.6,15.0,Mike Dunleavy,Republican,NaN
2,Arizona,13.1,9.0,Doug Ducey,Republican,NaN
3,Arkansas,23.6,24.0,Asa Hutchinson,Republican,NaN
4,California,9.0,-24.0,Gavin Newsom,Democratic,1.0


In [126]:
state_dataset = state_dataset.fillna(0)
state_dataset['climate_alliance'] = state_dataset['climate_alliance'].astype('bool')
state_dataset.head(10)

,state,CO2_per_capita,partisanship,governor,governor_party,climate_alliance
0,Alabama,23.2,27.0,Kay Ivey,Republican,False
1,Alaska,47.6,15.0,Mike Dunleavy,Republican,False
2,Arizona,13.1,9.0,Doug Ducey,Republican,False
3,Arkansas,23.6,24.0,Asa Hutchinson,Republican,False
4,California,9.0,-24.0,Gavin Newsom,Democratic,True
5,Colorado,15.9,-1.0,Jared Polis,Democratic,True
6,Connecticut,10.5,-11.0,Ned Lamont,Democratic,True
7,Delaware,13.8,-14.0,John C. Carney Jr.,Democratic,True
8,Florida,11.4,5.0,Ron DeSantis,Republican,False
9,Georgia,13.6,12.0,Brian Kemp,Republican,False


In [127]:
state_dataset.to_csv('/Users/joeskvarna/Desktop/Thesis/Data/state_dataset.csv', index = False)